In [1]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from chromedriver_py import binary_path   
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import pandas as pd

In [2]:


# Function to initialize Chrome WebDriver
def initialize_chrome():
    chrome_options = ChromeOptions()
    chrome_options.add_argument('log-level=3')
    chrome_options.add_argument('--ignore-certificate-errors-spki-list')
    chrome_options.add_argument('--ignore-ssl-errors')
    #chrome_options.add_argument('--headless')  # You can uncomment this if you want to run Chrome in headless mode
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(), options=chrome_options)
    driver.maximize_window()  # Maximize the window
    return driver
# Function to initialize Firefox WebDriver
def initialize_firefox():
    firefox_options = FirefoxOptions()
    firefox_options.headless = True  # Run Firefox in headless mode
    driver = webdriver.Firefox(options=firefox_options)
    driver.maximize_window()  # Maximize the window
    return driver
# Function to initialize WebDriver
def initialize_driver():
    try:
        driver = initialize_firefox()
        print("Firefox WebDriver initialized successfully.")
        return driver
    except Exception as e:
        print("Chrome WebDriver initialization failed:", e)
        print("Switching to chrome WebDriver...")
        return initialize_chrome()

# Function to wait for element visibility
def wait_for_element_visibility(driver, by, selector, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((by, selector))
    )

# Load the website
driver = initialize_driver()

driver.get("https://panchatantra.karnataka.gov.in/USER_MODULE/userLogin/loadHomePage")

Firefox WebDriver initialized successfully.


In [3]:

# Wait for the language selector element to become clickable
language_selector = wait_for_element_visibility(driver, By.XPATH, '//*[@id="homePageForm"]/section/div/div/div/div[2]/div/ul/li[3]/label')
# Click on the language selector element
language_selector.click()

In [21]:

# Wait for the element to become clickable
element = wait_for_element_visibility(driver, By.XPATH, '//*[@id="moduleLogoId5"]')

# Perform a right-click on the element
action = ActionChains(driver)
action.context_click(element).perform()

# Send the keyboard shortcut for opening the link in a new tab (Ctrl + Enter)
action.send_keys(Keys.ENTER).perform()

In [23]:
# Find the element
def get_dropdown_options(driver, XPATH):
 

    # Find all the options within the dropdown
    options = driver.find_elements(By.XPATH, f'{XPATH}/option[position() > 1]')

    # Iterate over the options and extract their text and values
    option_names = []
    option_values = []
    for option in options:
        option_text = option.text
        option_value = option.get_attribute("value")
        option_names.append(option_text)
        option_values.append(option_value)

    return option_names, option_values


# DISTRICT
district_names, district_values = get_dropdown_options(driver,'//*[@id="flt_zp__Id"]')

#select the first element in the district
css_selector_district = '[aria-labelledby="select2-flt_zp__Id-container"]' 

# Find the dropdown menu
dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_district)

#Click on the element to open the dropdown
ActionChains(driver).click(dropdown_menu).perform()

#Find the text box element
text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

# Clear any existing text in the text box
text_box.clear()

# Enter your desired text into the text box
text_box.send_keys(district_names[0])

#Press the Enter key to confirm the input
text_box.send_keys(Keys.ENTER)


#TALUK 
taluk_names, taluk_values = get_dropdown_options(driver, '//*[@id="flt_tp__Id"]')

css_selector_taluk = '[aria-labelledby="select2-flt_tp__Id-container"]' 

# Find the dropdown menu
dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_taluk)

#Click on the element to open the dropdown
ActionChains(driver).click(dropdown_menu).perform()

#Find the text box element
text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

# Clear any existing text in the text box
text_box.clear()

# Enter your desired text into the text box
text_box.send_keys(taluk_names[0])
       
#Press the Enter key to confirm the input
text_box.send_keys(Keys.ENTER)

#GRAM_PANCHAYAT

GP_names, GP_values = get_dropdown_options(driver, '//*[@id="flt_gp__Id"]')
#Select a Taluk one by one 
css_selector_gp = '[aria-labelledby="select2-flt_gp__Id-container"]' 

# Find the dropdown menu
dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_gp)

#Click on the element to open the dropdow
ActionChains(driver).click(dropdown_menu).perform()

#Find the text box element
text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

# Clear any existing text in the text box
text_box.clear()

# Enter your desired text into the text box
text_box.send_keys(GP_names[0])

#Press the Enter key to confirm the input
text_box.send_keys(Keys.ENTER)

 #CLICK ON THE SEARCH DATA 
Search_Button = driver.find_element(By.XPATH, '//*[@id="gp_dtl_searchBtn__ID"]')
              
#Click on the element to open the dropdown
ActionChains(driver).click(Search_Button).perform()

# Wait for the element to be visible
wait = WebDriverWait(driver, 10)
element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="count_cards__ID"]/div[4]/div/div/div/div[1]/p/span')))

# Adjust this value as needed
scroll_amount = 500

# Scroll the page down
for _ in range(scroll_amount // 100):
    driver.execute_script("window.scrollBy(0, 200);")
    time.sleep(5)  # Adjust the sleep time as needed

from selenium.common.exceptions import StaleElementReferenceException
try:
    # Click on the element
    element.click()

    # Wait for the new page to fully load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.number_of_windows_to_be(2))  # Wait for two windows (or tabs) to be open
    #Switch the WebDriver's focus to the new window or tab
    driver.switch_to.window(driver.window_handles[1])  # Assuming the new page opens in a new tab
except StaleElementReferenceException:
    # If the element is stale, re-locate it and try clicking again
    # Wait for the element to be visible
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="count_cards__ID"]/div[4]/div/div/div/div[1]/p/span')))
    # Click on the Total_Demand
    element.click()
    # Wait for the new page to fully load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.number_of_windows_to_be(2))  # Wait for two windows (or tabs) to be open
    #Switch the WebDriver's focus to the new window or tab
    driver.switch_to.window(driver.window_handles[1])  # Assuming the new page opens in a new tab



district_names, district_values = get_dropdown_options(driver,'//*[@id="flt_zp__Id"]')
taluk_names, taluk_values = get_dropdown_options(driver, '//*[@id="flt_tp__Id"]')
GP_names, GP_values = get_dropdown_options(driver, '//*[@id="flt_gp__Id"]')

In [25]:
from selenium.common.exceptions import NoSuchElementException
import re 
# Switch to the new tab
driver.switch_to.window(driver.window_handles[1])



css_selector_FY = '[aria-labelledby="select2-flt_finYear__Id-container"]'
              
              # Find the dropdown menu
dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_FY)

              #Click on the element to open the dropdown
ActionChains(driver).click(dropdown_menu).perform()

              #Find the text box element
text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

              # Clear any existing text in the text box
text_box.clear()

              # Enter your desired text into the text box
text_box.send_keys("2023-24")

              #Press the Enter key to confirm the input
text_box.send_keys(Keys.ENTER) 
# Define the XPATH pattern
xpath_pattern = '//*[@id="gp_rev_collection_row__ID"]/div[{}]/div'

#Initialize the index
index = 1

# Initialize lists to store the values
value1_list = []
value2_list = []

# Loop to find and click on the elements matching the pattern
while True:
    # Construct the XPATH with the current index
    xpath = xpath_pattern.format(index)
    
    # Try to find the element
    try:
        element = driver.find_element(By.XPATH, xpath)
        
       # Get the value of the onclick attribute
        onclick_value = element.get_attribute("onclick")
        
        # Use regular expressions to extract the desired string
 
        match = re.search(r'viewRevenueDetailsByVillageId\(this,"(\d+)","(\d+)","([^"]+)"\)', onclick_value)           

        num_matches = re.finditer(r'viewRevenueDetailsByVillageId\(this,"(\d+)","(\d+)","([^"]+)"\)', onclick_value)           

        # Extract the values
        if match:
           value1 = match.group(2)
           value2 = match.group(3)

           print("First Value:", value1)
           print("Second Value:", value2)

        for i in num_matches:
            value1_list.append(value1)
            value2_list.append(value2)
        # Wait for some time to ensure the page is loaded (you can adjust the time as needed)
        time.sleep(2)
        
        # Move to the next index
        index += 1
        
    except NoSuchElementException:
        # If no element is found, break out of the loop
        break 

First Value: 1501001009001
Second Value: KABBALAGERI
First Value: 1501001009003
Second Value: ADAGAL


In [29]:
PANCHMITRA_KARNATAKA_data =[]
driver.switch_to.window(driver.window_handles[0])  # Assuming the new page opens in a new tab


css_selector_district = '[aria-labelledby="select2-flt_zp__Id-container"]' 

for i in range(len(district_names)): 

    # Find the dropdown menu
    dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_district)

    #Click on the element to open the dropdown
    ActionChains(driver).click(dropdown_menu).perform()

    #Find the text box element
    text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

    # Clear any existing text in the text box
    text_box.clear()

    # Enter your desired text into the text box
    text_box.send_keys(district_names[i])

    #Press the Enter key to confirm the input
    text_box.send_keys(Keys.ENTER)
    
    #Select a Taluk one by one 
    css_selector_taluk = '[aria-labelledby="select2-flt_tp__Id-container"]' 

    for j in range(len(taluk_names)): 

         # Find the dropdown menu
         dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_taluk)

         #Click on the element to open the dropdown
         ActionChains(driver).click(dropdown_menu).perform()

         #Find the text box element
         text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input')

         # Clear any existing text in the text box
         text_box.clear()

         # Enter your desired text into the text box
         text_box.send_keys(taluk_names[j])

         
         #Press the Enter key to confirm the input
         text_box.send_keys(Keys.ENTER)

         #################################################
         #GET PANCHAYAT NAMES AND ITS VALUES FOR A TALUK UNDER A DISTRICT 

         css_selector_gp = '[aria-labelledby="select2-flt_gp__Id-container"]' 

         for k in range(len(GP_names)): 

              # Find the dropdown menu
              #dropdown_menu = driver.find_element(By.CSS_SELECTOR, css_selector_gp)
              dropdown_menu = driver.find_element(By.ID, "select2-flt_gp__Id-container")

              #Click on the element to open the dropdown
              ActionChains(driver).click(dropdown_menu).perform()

              #Find the text box element
              text_box = driver.find_element(By.XPATH, '/html/body/span/span/span[1]/input' )

              # Clear any existing text in the text box
              text_box.clear()

              # Enter your desired text into the text box
              text_box.send_keys(GP_names[k])

              #Press the Enter key to confirm the input
              text_box.send_keys(Keys.ENTER)

              #CLICK ON THE SEARCH DATA 
              Search_Button = driver.find_element(By.XPATH,  '//*[@id="searchAwardList"]').click()
            
              # Define the XPATH pattern
              xpath_pattern = '//*[@id="gp_rev_collection_row__ID"]/div[{}]/div'
                
                
              # Initialize the index
              index = 1

              # Loop to find and click on the elements matching the pattern
              while True:
                 # Construct the XPATH with the current index
                    xpath = xpath_pattern.format(index)
    
                    # Try to find the element
                    try:
                        element = driver.find_element(By.XPATH, xpath)
        
                        # Get the value of the onclick attribute
                        onclick_value = element.get_attribute("onclick")
        
                        # Use regular expressions to extract the desired string
 
                        match = re.search(r'viewRevenueDetailsByVillageId\(this,"(\d+)","(\d+)","([^"]+)"\)', onclick_value)           

                        num_matches = re.finditer(r'viewRevenueDetailsByVillageId\(this,"(\d+)","(\d+)","([^"]+)"\)', onclick_value)           

                    # Extract the values
                        if match:
                           value1 = match.group(2)
                           value2 = match.group(3)

                        # Move to the next index
                        index += 1
        
                    except NoSuchElementException:
                     # If no element is found, break out of the loop
                           break
            


In [9]:
df = pd.DataFrame(PANCHMITRA_KARNATAKA_data)
df.head()

,District Name,District Value,Taluk Name,Taluk Value,GP Name,GP Value
0,BAGALKOTE,1501,BADAMI,1501001,ADAGALL,1501001009
1,BAGALKOTE,1501,BADAMI,1501001,ALUR-SK,1501001036
2,BAGALKOTE,1501,BADAMI,1501001,ANAVAL,1501001024
3,BAGALKOTE,1501,BADAMI,1501001,BELOOR,1501001015
4,BAGALKOTE,1501,BADAMI,1501001,CHOLACHAGUDD,1501001010


In [10]:
# Define the file path where you want to save the CSV file
file_path = "C:\\Users\\niranjan.kumar\\Desktop\Property_Tax\\Raw_Data\\PANCHMITRA_KARNATAKA.csv"

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print("DataFrame saved successfully to:", file_path)

DataFrame saved successfully to: C:\Users\niranjan.kumar\Desktop\Property_Tax\Raw_Data\PANCHMITRA_KARNATAKA.csv


<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\niranjan.kumar\AppData\Local\Temp\ipykernel_22864\1506352004.py:2: SyntaxWarning: invalid escape sequence '\P'
  file_path = "C:\\Users\\niranjan.kumar\\Desktop\Property_Tax\\Raw_Data\\PANCHMITRA_KARNATAKA.csv"


In [72]:
import http.client
import ssl

conn = http.client.HTTPSConnection("panchatantra.karnataka.gov.in", context=ssl._create_unverified_context())

fin_year_list = ["2019-2020","2020-2021", "2021-2022", "2022-2023", "2023-2024"]

# Create an empty DataFrame to store the results
combined_df = pd.DataFrame()


master_df = pd.DataFrame() 
for j in range(len(value1_list)): 
       
  temp_id =  value1_list[j]

  temp_name = value2_list[j]
  
  print(temp_id, temp_name)

  for i in fin_year_list:

    payload = "{\"gp_id\":\"1501001009\",\"village_id\":\" "+temp_id +" \",\"fin_year\":\" " + i + " \"}"  # Concatenate the value of i into the payload
    headers = {
     'Accept': '*/*',
     'Accept-Language': 'en-US,en;q=0.9',
     'Connection': 'keep-alive',
     'Content-Type': 'application/json; charset=UTF-8',
     'Cookie': 'JSESSIONID=269F574E9B730E827C79D453605D69B1; HstCfa4757307=1711786440923; HstCmu4757307=1711786440923; HstCnv4757307=8; HstCns4757307=9; HstCla4757307=1712168169883; HstPn4757307=5; HstPt4757307=27',
     'Origin': 'https://panchatantra.karnataka.gov.in',
     'Referer': 'https://panchatantra.karnataka.gov.in/USER_MODULE/userLogin/viewPropertiesDetailsByVillage',
     'Sec-Fetch-Dest': 'empty',
     'Sec-Fetch-Mode': 'cors',
     'Sec-Fetch-Site': 'same-origin',
     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
     'X-Requested-With': 'XMLHttpRequest',
     'sec-ch-ua': '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
     'sec-ch-ua-mobile': '?0',
     'sec-ch-ua-platform': '"Windows"'
    }
    conn.request("POST", "/USER_MODULE/gpDashboard/getOperationWebService?serviceName=getGpPropertiesDetailsByVillageId&serviceType=MASTER", payload, headers)
    res = conn.getresponse()
    data = res.read()
    data_new = data.decode("utf-8")

    print(data_new)

    import pandas as pd
    import json
    from pandas import json_normalize 



  # Flatten the nested dictionary
    data_dict = json.loads(data_new)
# Extract the responseData from the dictionary
    response_data = data_dict.get("responseData")

# Parse the responseData as JSON
    response_data_dict = json.loads(response_data)

# Flatten the nested dictionary
    flat_data = json_normalize(response_data_dict)

# Create a DataFrame from the flattened dictionary
    df = pd.DataFrame(flat_data)

    df["year"] = i

    df["Village_ID"] =  temp_id
    
    df["Village_NAME"]= temp_name

    #print(df.head())
  
    # Append the DataFrame to the combined DataFrame
    combined_df = pd.concat([combined_df, df], ignore_index=True)

    combined_df["Property_Id"] =  'p_'+ combined_df["property_id"].astype(str)

    #combined_df = pd.concat([combined_df, df.astype({"property_id": str})], ignore_index=True)



master_df = pd.concat([master_df, combined_df], ignore_index=True)

print(master_df.head())

# Define the file path for the CSV file
csv_file_path = "C:\\Users\\niranjan.kumar\\documents\\karnataka_panch_test.csv"

# Write DataFrame to CSV file
# Write DataFrame to CSV file with float_format='{:f}' to prevent truncation
master_df.to_csv(csv_file_path, index=False)

print("CSV file has been created successfully at:", csv_file_path)




1501001009001 KABBALAGERI
{"status":1000,"isValidated":false,"serviceStatus":false,"responseData":"[{\"total_balance\":859.88,\"property_number\":\"63\",\"arrears\":0,\"total_demand\":859.88,\"property_owner_name\":\"ಬಸಪ್ಪ ಭೀಮಪ್ಪ ತಳ್ಳಿಕೇರಿ\",\"total_demand_collection\":0,\"current_demand\":859.88,\"property_id\":\"150100100900100063\"},{\"total_balance\":1048.04,\"property_number\":\"361\",\"arrears\":0,\"total_demand\":1048.04,\"property_owner_name\":\"ಸೋಮಪ್ಪ ಶಿದ್ದಪ್ಪ ನೀರಲಕೇರಿ\",\"total_demand_collection\":0,\"current_demand\":1048.04,\"property_id\":\"150100100900100373\"},{\"total_balance\":6.2,\"property_number\":\"324\",\"arrears\":0,\"total_demand\":6.2,\"property_owner_name\":\"ಸುರೇಶ ಬಾಳಾಜಿ ಕುಲಕರ್ಣಿ\",\"total_demand_collection\":0,\"current_demand\":6.2,\"property_id\":\"150100100900100334\"},{\"total_balance\":351.08,\"property_number\":\"364\",\"arrears\":0,\"total_demand\":351.08,\"property_owner_name\":\"ತಾಜಬಾಸಾಬ ಇಮಾಮಸಾಬ ಪಠಾಣ\",\"total_demand_collection\":0,\"current_demand\